In [2]:
import pandas as pd
import numpy as np

In [3]:
data_1 = pd.read_csv('data/Mistral/temp_1/data_mistral_complete_0.tsv', sep='\t')
data_2 = pd.read_csv('data/Mistral/temp_1/data_mistral_complete_1.tsv', sep='\t')
data_3 = pd.read_csv('data/Mistral/temp_1/data_mistral_complete_2.tsv', sep='\t')
data_4 = pd.read_csv('data/Mistral/temp_1/data_mistral_complete_3.tsv', sep='\t')
data_5 = pd.read_csv('data/Mistral/temp_1/data_mistral_complete_4.tsv', sep='\t')


In [4]:
print((data_1 == data_2).sum().sum())
print((data_2 == data_3).sum().sum())
print((data_3 == data_4).sum().sum())
print((data_4 == data_5).sum().sum())

5100
5100
5100
5100


In [3]:
new_data = data_1.copy()
target_columns = ['far_right_EN', 'far_right_PL', 'far_right_RU', 'mod_cons_EN', 'mod_cons_PL', 'mod_cons_RU',
       'prog_left_EN', 'prog_left_PL', 'prog_left_RU', 'centrist_EN', 'centrist_PL', 'centrist_RU']
for target_column in target_columns:
    new_data[f'Qwen_{target_column}_answer'] = np.nan
new_data.head()

,Text_EN,Text_PL,Text_RU,Translation_quality,Qwen_far_right_EN_answer,Qwen_far_right_PL_answer,Qwen_far_right_RU_answer,Qwen_mod_cons_EN_answer,Qwen_mod_cons_PL_answer,Qwen_mod_cons_RU_answer,Qwen_prog_left_EN_answer,Qwen_prog_left_PL_answer,Qwen_prog_left_RU_answer,Qwen_centrist_EN_answer,Qwen_centrist_PL_answer,Qwen_centrist_RU_answer
1,<user> <user> <user> <user> Um the Kurds are h...,"<user> <user> <user> <user> Um, Kurdowie pomag...","<user> <user> <user> <user> Эм, курды помогают...",1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,<user> This earpiece too plus a wire on his sl...,<user> Ta słuchawka też plus przewód na rękawa...,<user> Этот наушник тоже плюс провод на рукава...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,<user> <user> Please lost you yelling insanely...,"<użytkownik> <użytkownik> Proszę, zgubcie się,...","<user> <user> Пожалуйста, потеряйтесь, безумно...",1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"<user> <user> <user> Just take the L g, or fin...",<user> <user> <user> Po prostu weź L g lub zna...,"<user> <user> <user> Просто возьми L g, или на...",0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Elections have consequences 101: - Republican...,Wybory mają konsekwencje 101: - Republikańscy ...,Выборы имеют последствия 101: - избиратели-рес...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
colnames = [col for col in new_data.columns if col.endswith('_answer')]
# For each column that ends with "_answer"
valid = 0
for index, row in new_data.iterrows():
    for i in range(len(colnames)):
        column_sum = data_1.loc[index, colnames[i]] + data_2.loc[index, colnames[i]] + data_3.loc[index, colnames[i]] + data_4.loc[index, colnames[i]] + data_5.loc[index, colnames[i]]
        
        if (column_sum >=4 or column_sum <= 1):
            if row['Translation_quality'] == 1.0:
                valid += 1
            if column_sum >= 4:
                new_data.loc[index, colnames[i]] = 1
            else:
                new_data.loc[index, colnames[i]] = 0

print(valid, valid/(new_data[new_data['Translation_quality'] == 1.0].shape[0]*12), (new_data[new_data['Translation_quality'] == 1.0].shape[0]*12) - valid)

3564 1.0 0


In [7]:
new_data.to_parquet('data/Gemma/temp_1/data_aggregated.parquet', engine='pyarrow')